# 0.Setup

In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (9, 4)

import normet as nm

In [2]:
#Optional
#Enable concise logging (scrolling box is handy in notebooks):
from normet.utils.logging import enable_default_logging, get_logger
enable_default_logging(level="INFO")
log = get_logger("tutorial.model_training")

# 1. Data: load

In [3]:
df=pd.read_csv(r'data_MY1_data.csv',parse_dates=['date'],index_col='date')

In [4]:
df.columns

Index(['O3', 'NO', 'NO2', 'NOXasNO2', 'SO2', 'CO', 'PM10', 'NV10', 'V10',
       'PM2.5', 'NV2.5', 'V2.5', 'ETHANE', 'ETHENE', 'ETHYNE', 'PROPANE',
       'PROPENE', 'iBUTANE', 'nBUTANE', '1BUTENE', 't2BUTENE', 'c2BUTENE',
       'iPENTANE', 'nPENTANE', 't2PENTEN', '1PENTEN', '2MEPENT', 'ISOPRENE',
       'nHEXANE', 'nHEPTANE', 'iOCTANE', 'nOCTANE', 'BENZENE', 'TOLUENE',
       'ETHBENZ', 'mpXYLENE', 'oXYLENE', '124TMB', '135TMB', 'wd', 'ws',
       'temp', 'AT10', 'AP10', 'AT2.5', 'AP2.5', 'site', 'code', 'latitude',
       'longitude', 'location_type', 'Ox', 'NOx', 'u10', 'v10', 'd2m', 't2m',
       'blh', 'sp', 'ssrd', 'tcc', 'tp', 'rh2m', 'lat', 'lon'],
      dtype='object')

In [5]:
# Prepare data, including adding time variables, training/testing split
df_prep=nm.prepare_data(df, value='PM2.5', feature_names=['u10', 'v10', 'd2m', 't2m',
       'blh', 'sp', 'ssrd', 'tcc', 'tp', 'rh2m'])

INFO | normet.normet.utils.prepare | Dropped 2411 rows with NA in target.
INFO | normet.normet.utils.prepare | Prepared data: 6373 rows, 17 columns


In [6]:
?nm.prepare_data

Signature:
nm.prepare_data(
    df: 'pd.DataFrame',
    value: 'str',
    feature_names: 'List[str]',
    na_rm: 'bool' = True,
    split_method: 'str' = 'random',
    fraction: 'float' = 0.75,
    seed: 'int' = 7654321,
) -> 'pd.DataFrame'
Docstring:
Clean, validate, and split the input DataFrame in a single pipeline.

Steps:
  1) Ensure a datetime column named ``date`` is present.
  2) Validate target and features.
  3) Impute/drop missing values.
  4) Add derived date variables (unix, julian day, weekday, hour).
  5) Split into training/testing sets.

Parameters
----------
df : pandas.DataFrame
    Raw input dataset containing at least the target column and date/time info.
value : str
    Target column name in ``df``.
feature_names : list of str
    Predictor variable names to keep (must exist in ``df``).
na_rm : bool, default True
    If True, drop rows where the target is NA. Also imputes other NAs.
split_method : {"random","ts","season","month"}, default "random"
    Train/test s

In [7]:
df_prep.columns

Index(['blh', 'd2m', 'rh2m', 'sp', 'ssrd', 't2m', 'tcc', 'tp', 'u10', 'v10',
       'date', 'value', 'date_unix', 'day_julian', 'weekday', 'hour', 'set'],
      dtype='object')

In [8]:
df_prep.to_csv('data_df_prep.csv',index=False)

# 2.Training model with FLAML backend (lightweight, recommended default)

## 2.1 Quick start

In [9]:
target = 'value'
features = ['u10', 'v10', 'd2m', 't2m',
       'blh', 'sp', 'ssrd', 'tcc', 'tp', 'rh2m','date_unix', 'day_julian', 'weekday',
       'hour']

In [10]:
#Use a single estimator (e.g., LightGBM) for a quick baseline.
#Below we use LightGBM with a small time budget.
flaml_quick_cfg = {
    "time_budget": 90,          # seconds for the search
    "metric": "r2",             # optimize R^2 (use "mae"/"mse" if preferred)
    "estimator_list": ["lgbm"], # single estimator keeps things fast
    "task": "regression",
    "eval_method": "auto",
    "save_model": True,
    "folder_path":'.',
    "filename": "automl"
}

model_flaml = nm.train_model(
    df=df_prep,
    value= target,
    backend="flaml",
    variables=features,
    model_config=flaml_quick_cfg,
    seed=42,
    verbose=True,
)

INFO | normet.normet.backends.flaml_backend | Training FLAML AutoML: X shape=(4780, 14), target='value'
INFO | normet.normet.backends.flaml_backend | FLAML best_estimator=lgbm | best_config={'n_estimators': 1149, 'num_leaves': 14, 'min_child_samples': 15, 'learning_rate': np.float64(0.2061003131290804), 'log_max_bin': 10, 'colsample_bytree': np.float64(0.9510000624084615), 'reg_alpha': np.float64(1.162262163142946), 'reg_lambda': np.float64(0.0034651839263365124)}
INFO | normet.normet.backends.flaml_backend | Saved FLAML model to automl.joblib


In [11]:
model_flaml

,time_budget,-1
,task,'classification'
,n_jobs,-1
,eval_method,'auto'
,split_ratio,0.1
,n_splits,5
,auto_augment,True
,metric,'auto'
,estimator_list,'auto'
,log_file_name,''
,max_iter,None


# 2.2 Save/load FLAML model

In [12]:
# You can still save model if you didn't set save model as True in the model config during model training
nm.save_model(model_flaml,folder_path='.',filename='automl.joblib')

INFO | normet.normet.backends.flaml_backend | Saved FLAML model to automl.joblib


'automl.joblib'

In [13]:
?nm.save_model

Signature:
nm.save_model(
    model: 'object',
    folder_path: 'Union[str, Path]' = '.',
    filename: 'str' = 'automl.joblib',
) -> 'str'
Docstring:
Save a trained model by delegating to the appropriate backend saver.

Parameters
----------
model : object
    Trained model with attribute ``backend`` in {"flaml", "h2o"}.
folder_path : str | Path, default="."
    Destination directory. Created if it does not exist.
filename : str, default="automl.joblib"
    Output name:
      - FLAML: exact filename, must end with ``.joblib`` or ``.pkl``.
      - H2O:   desired artifact name; handled by renaming after save.

Returns
-------
str
    Path to the saved artifact:
      - FLAML: full path to the ``.joblib``/``.pkl`` file.
      - H2O:   full path to the renamed model file.

Raises
------
AttributeError
    If model does not define a ``backend`` attribute.
TypeError
    If backend is unsupported.
File:      ~/anaconda3/envs/normet/lib/python3.11/site-packages/normet/model/io.py
Type:      f

In [14]:
model_flaml=nm.load_model(folder_path='.',filename='automl.joblib')

INFO | normet.normet.backends.flaml_backend | Loaded FLAML model from automl.joblib


# 2.3 Model performance_FLAML

In [15]:
modStat = nm.modStats(df_prep, model_flaml)

In [16]:
modStat

,n,FAC2,MB,MGE,RMSE,NMB,NMGE,COE,IOA,r,p_level,R2,set
0,4780,0.990153,-0.000040,0.253551,0.330469,-0.000004,0.027825,0.953687,0.976843,0.999207,***,0.998415,training
1,1593,0.888050,0.153261,2.052048,2.931742,0.016659,0.223050,0.626259,0.813130,0.929289,***,0.863578,testing
2,6373,0.964639,0.038280,0.703104,1.493437,0.004191,0.076975,0.871680,0.935840,0.983044,***,0.966376,all


# 2.4 Partial Dependency_FLAML

In [17]:
pdp_value=nm.pdp(df_prep,model_flaml)

In [18]:
pdp_value.head()

,variable,value,pdp_mean,pdp_std
0,blh,32.616693,19.345874,9.574783
1,blh,71.202291,18.257101,9.641589
2,blh,109.787888,17.079692,9.567257
3,blh,148.373486,14.047617,7.889420
4,blh,186.959084,12.244408,7.785849


In [19]:
df_prep.columns

Index(['blh', 'd2m', 'rh2m', 'sp', 'ssrd', 't2m', 'tcc', 'tp', 'u10', 'v10',
       'date', 'value', 'date_unix', 'day_julian', 'weekday', 'hour', 'set'],
      dtype='object')

In [20]:
nm.pdp(df_prep,model_flaml,var_list=['blh','rh2m'])

,variable,value,pdp_mean,pdp_std
0,blh,32.616693,19.345874,9.574783
1,blh,71.202291,18.257101,9.641589
2,blh,109.787888,17.079692,9.567257
3,blh,148.373486,14.047617,7.889420
4,blh,186.959084,12.244408,7.785849
...,...,...,...,...
95,rh2m,93.271444,8.961660,7.744341
96,rh2m,94.519172,9.170470,7.782532
97,rh2m,95.766900,9.505157,7.829908
98,rh2m,97.014628,9.367317,7.753671


# 3.Training model with H2O backend (heavier, Java required)

## 3.1 Quick config

In [21]:
target = 'value'
features = ['u10', 'v10', 'd2m', 't2m',
       'blh', 'sp', 'ssrd', 'tcc', 'tp', 'rh2m','date_unix', 'day_julian', 'weekday',
       'hour']

h2o_quick_cfg = {
    "max_models": 8,
    "include_algos": ["GBM"],  # keep small for speed; expand later
    "sort_metric": "RMSE",
    "max_mem_size": "8G",      # adjust to your machine/data
    "save_model": True,
    "folder_path":'.',
    "filename":"automl"
}


model_h2o = nm.train_model(
    df=df_prep,
    value=target,
    backend="h2o",
    variables=features,
    model_config=h2o_quick_cfg,
    seed=42,
    verbose=False,)

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,4 mins 46 secs
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,5 months and 21 days
H2O_cluster_name:,H2O_from_python_n94921cs_um8ihg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.071 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,7
H2O_cluster_status:,"locked, healthy"


INFO | normet.normet.backends.h2o_backend | H2O cluster up | version=None | nthreads=? | mem=8G
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


In [22]:
model_h2o

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_2_20250917_163906


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    144                144                         170038                 10           10           10            25            237           89.3125

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1.3658872273886002
RMSE: 1.168711781145634
MAE: 0.8650958923159253
RMSLE: NaN
Mean Residual Deviance: 1.3658872273886002

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 8.589262525970822
RMSE: 2.9307443638043256
MAE: 1.998121364481201
RMSLE: NaN
Mean Residual Deviance: 8.589262525970822

Cross-Validation Metrics Summary: 
                        mean      sd         cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ---------  ------------  ------------  ------------  ------------  ------------
aic                     nan       0          nan           nan           nan           nan           nan
loglikelihood           nan       0          nan           nan           nan           nan           nan
mae                     1.99812   0.0528831  1.99892       2.02293       2.0682        1.97354       1.92702
mean_residual_deviance  8.58926   0.6306     9.18404       8.4784        9.29273       8.10689       7.88426
mse                     8.58926   0.6306     9.18404       8.4784        9.29273       8.10689       7.88426
r2                      0.872521  0.0073285  0.865921      0.870267      0.865808      0.880224      0.880382
residual_deviance       8.58926   0.6306     9.18404       8.4784        9.29273       8.10689       7.88426
rmse                    2.92917   0.107479   3.03052       2.91177       3.0484        2.84726       2.80789
rmsle                   nan       0          nan           nan           nan           nan           nan

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse       training_mae        training_deviance
---  -------------------  ----------  -----------------  ------------------  ------------------  -------------------
     2025-09-17 16:39:13  1.010 sec   0.0                8.206234628707291   5.474664040669193   67.34228678139468
     2025-09-17 16:39:13  1.028 sec   5.0                5.83831213143245    3.8835318260284524  34.08588854403132
     2025-09-17 16:39:13  1.046 sec   10.0               4.367820596570432   2.9034753837018705  19.077856763824883
     2025-09-17 16:39:13  1.061 sec   15.0               3.4818413049519683  2.3349531707077222  12.123218872869625
     2025-09-17 16:39:13  1.076 sec   20.0               2.9559543185838635  2.0045669553840386  8.737665933554593
     2025-09-17 16:39:13  1.091 sec   25.0               2.615763851650526   1.7935225121817204  6.842220527601595
     2025-09-17 16:39:13  1.106 sec   30.0               2.3602398254843253  1.6476785901435125  5.570732033802278
     2025-09-17 16:39:13  1.119 sec   35.0               2.2005216222948394  1.5521327791291302  4.842295410187111
     2025-09-17 16:39:13  1.135 sec   40.0               2.071478605283562   1.472422849769591   4.29102361214753
     2025-09-17 16:39:13  1.150 sec   45.0               1.963189871534841   1.39875936434764    3.8541144716969855
---  ---                  ---         ---                ---                 ---                 ---
     2025-09-17 16:39:14  1.298 sec   100.0              1.4029349452265631  1.0264408625038848  1.9682264605378599
     2025-09-17 16:39:14  1.310 sec   105.0              1.3712223920395326  1.0056641323187985  1.8802508484306177
     2025-09-17 16:39:14  1.323 sec   110.0   

# 3.2 Save/load H2O model

In [23]:
# You can still save model if you didn't set save model as True in the model config during model training
nm.save_model(model_h2o,folder_path='.',filename='automl')

'automl'

In [24]:
model_h2o=nm.load_model(folder_path='.',backend='h2o',filename='automl')

INFO | normet.normet.backends.h2o_backend | Loaded H2O model from automl


In [25]:
model_h2o

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_2_20250917_163906


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    144                144                         170038                 10           10           10            25            237           89.3125

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1.3658872273886002
RMSE: 1.168711781145634
MAE: 0.8650958923159253
RMSLE: NaN
Mean Residual Deviance: 1.3658872273886002

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 8.589262525970822
RMSE: 2.9307443638043256
MAE: 1.998121364481201
RMSLE: NaN
Mean Residual Deviance: 8.589262525970822

Cross-Validation Metrics Summary: 
                        mean      sd         cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ---------  ------------  ------------  ------------  ------------  ------------
aic                     nan       0          nan           nan           nan           nan           nan
loglikelihood           nan       0          nan           nan           nan           nan           nan
mae                     1.99812   0.0528831  1.99892       2.02293       2.0682        1.97354       1.92702
mean_residual_deviance  8.58926   0.6306     9.18404       8.4784        9.29273       8.10689       7.88426
mse                     8.58926   0.6306     9.18404       8.4784        9.29273       8.10689       7.88426
r2                      0.872521  0.0073285  0.865921      0.870267      0.865808      0.880224      0.880382
residual_deviance       8.58926   0.6306     9.18404       8.4784        9.29273       8.10689       7.88426
rmse                    2.92917   0.107479   3.03052       2.91177       3.0484        2.84726       2.80789
rmsle                   nan       0          nan           nan           nan           nan           nan

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse       training_mae        training_deviance
---  -------------------  ----------  -----------------  ------------------  ------------------  -------------------
     2025-09-17 16:39:13  1.010 sec   0.0                8.206234628707291   5.474664040669193   67.34228678139468
     2025-09-17 16:39:13  1.028 sec   5.0                5.83831213143245    3.8835318260284524  34.08588854403132
     2025-09-17 16:39:13  1.046 sec   10.0               4.367820596570432   2.9034753837018705  19.077856763824883
     2025-09-17 16:39:13  1.061 sec   15.0               3.4818413049519683  2.3349531707077222  12.123218872869625
     2025-09-17 16:39:13  1.076 sec   20.0               2.9559543185838635  2.0045669553840386  8.737665933554593
     2025-09-17 16:39:13  1.091 sec   25.0               2.615763851650526   1.7935225121817204  6.842220527601595
     2025-09-17 16:39:13  1.106 sec   30.0               2.3602398254843253  1.6476785901435125  5.570732033802278
     2025-09-17 16:39:13  1.119 sec   35.0               2.2005216222948394  1.5521327791291302  4.842295410187111
     2025-09-17 16:39:13  1.135 sec   40.0               2.071478605283562   1.472422849769591   4.29102361214753
     2025-09-17 16:39:13  1.150 sec   45.0               1.963189871534841   1.39875936434764    3.8541144716969855
---  ---                  ---         ---                ---                 ---                 ---
     2025-09-17 16:39:14  1.298 sec   100.0              1.4029349452265631  1.0264408625038848  1.9682264605378599
     2025-09-17 16:39:14  1.310 sec   105.0              1.3712223920395326  1.0056641323187985  1.8802508484306177
     2025-09-17 16:39:14  1.323 sec   110.0   

# 3.3 Model performance_H2O

In [26]:
modStat = nm.modStats(df_prep, model_h2o)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [27]:
modStat

,n,FAC2,MB,MGE,RMSE,NMB,NMGE,COE,IOA,r,p_level,R2,set
0,4780,0.968992,-0.000336,0.865096,1.168712,-0.000037,0.094937,0.841982,0.920991,0.990210,***,0.980516,training
1,1593,0.904403,0.072883,2.072402,2.958598,0.007922,0.225263,0.622552,0.811276,0.928224,***,0.861600,testing
2,6373,0.952852,0.017966,1.166875,1.792331,0.001967,0.127747,0.787039,0.893520,0.975859,***,0.952301,all


# 3.4 Partial Dependency_H2O

In [28]:
pdp_value=nm.pdp(df_prep,model_h2o)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |█████████████████

In [29]:
pdp_value.head()

,variable,value,pdp_mean,pdp_std
0,blh,11.407354,18.165007,8.524272
1,blh,156.247874,14.033806,7.425718
2,blh,301.088394,10.045159,5.802106
3,blh,445.928914,8.622272,5.030158
4,blh,590.769434,8.185836,4.807409


In [30]:
pdp_value['variable'].unique()

array(['blh', 'date_unix', 'u10', 'd2m', 'day_julian', 'sp', 'v10', 't2m',
       'weekday', 'hour', 'rh2m', 'tcc', 'ssrd', 'tp'], dtype=object)

In [31]:
nm.pdp(df_prep,model_h2o,var_list=['blh','rh2m'])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |████████████████████████████████████████████████| (done) 100%
PartialDependencePlot progress: |████████████████████████████████████████████████| (done) 100%


,variable,value,pdp_mean,pdp_std
0,blh,11.407354,18.165007,8.524272
1,blh,156.247874,14.033806,7.425718
2,blh,301.088394,10.045159,5.802106
3,blh,445.928914,8.622272,5.030158
4,blh,590.769434,8.185836,4.807409
5,blh,735.609954,7.702386,4.774138
6,blh,880.450474,7.396100,4.649164
7,blh,1025.290994,7.107440,4.423848
8,blh,1170.131514,7.093472,4.379220
9,blh,1314.972034,6.701338,4.373802
